In [1]:
import random
import turtle as t
alex = t.Turtle()
wn = t.Screen()

def init_positions(n_cells, n_veh):
    loc_c = [0] * n_cells
    total = 0
    while total != n_veh:
        index = random.randrange(0, n_cells - 1)
        if loc_c[index] == 0:
            loc_c[index] = 1
            total += 1
    return loc_c


def init_speeds(loc_c):
    spd_c = [0] * len(loc_c)
    for i in range(len(loc_c)):
        if loc_c[i] == 1:
            spd_c[i] = random.randrange(0, 6)
    return spd_c


def calc_accel(loc_c, spd_c):
    spd_n = [0] * len(loc_c)
    for i in range(len(loc_c)):
        if loc_c[i] == 1:
            if spd_c[i] < 5:
                spd_n[i] = spd_c[i] + 1
            else:
                spd_n[i] = spd_c[i]
    return spd_n


def calc_dist(loc_c):
    dis_c = [0] * len(loc_c)
    for i, v in enumerate(loc_c):
        if v:
            count = 1
            index = (i + 1) % len(loc_c)
            while not loc_c[index]:
                count += 1
                index = (index + 1) % len(loc_c)
            dis_c[i] = count
    return dis_c


def calc_braking(loc_c, spd_n, dis_c):
    spd_b = [0] * len(loc_c)
    for i in range(len(spd_n)):
        if loc_c[i] == 1:
            if spd_n[i] >= dis_c[i]:
                spd_b[i] = dis_c[i] - 1
            else:
                spd_b[i] = spd_n[i]
    return spd_b


def calc_reaction(loc_c, spd_b, p):
    spd_a = [0] * len(loc_c)
    for i in range(len(loc_c)):
        if loc_c[i] == 1:
            if random.random() <= p and spd_b[i] > 0:
                spd_a[i] = spd_b[i]-1
            else:
                spd_a[i] = spd_b[i]
    return spd_a


def calc_loc_spd(loc_c,spd_a):
    loc_n = [0]*len(loc_c)
    spd_n = [0]*len(spd_a)
    for i,v in enumerate(loc_c):
        if v == 1:
            loc_n[(i+spd_a[i]) % len(loc_c)] = v
            spd_n[(i+spd_a[i]) % len(spd_a)] = spd_a[i]
    return loc_n, spd_n


def simulate_flow(rho, p, n_cells, t_steps):
    n_veh = int(n_cells * rho)
    loc_c = init_positions(n_cells, n_veh)
    spd_c = init_speeds(loc_c)
    loc_l = [loc_c]
    spd_l = [spd_c]
    for t_step in range(t_steps):
        spd_n = calc_accel(loc_c, spd_c)
        dis_c = calc_dist(loc_c)
        spd_n = calc_braking(loc_c, spd_n, dis_c)
        spd_n = calc_reaction(loc_c, spd_n, p)
        loc_n, spd_n = calc_loc_spd(loc_c, spd_n)
        loc_l.append(loc_n)
        spd_l.append(spd_n)
        loc_c = loc_n
        spd_c = spd_n
    return loc_l, spd_l


def plot_flow(rho, p, n_cells, t_steps):
    loc_lists, spd_lists = simulate_flow(rho, p, n_cells, t_steps)
    x_min = 0
    x_max = n_cells
    y_max = t_steps
    y_min = 0
    wn.setworldcoordinates(x_min, y_min, x_max, y_max)
    wn.tracer(0)
    alex.up()
    alex.goto(x_min, y_max)
    for j in range(t_steps):
        loc_l = loc_lists[j]
        spd_l = spd_lists[j]
        for i in range(len(loc_l)):
            if loc_l[i] == 1:
                if spd_l[i] == 1:
                    alex.goto(i, t_steps - j)
                    alex.dot('orange')
                elif spd_l[i] == 2:
                    alex.goto(i, t_steps - j)
                    alex.dot('yellow')
                elif spd_l[i] == 3:
                    alex.goto(i, t_steps - j)
                    alex.dot('blue')
                elif spd_l[i] == 4:
                    alex.goto(i, t_steps - j)
                    alex.dot('purple')
                elif spd_l[i] == 5:
                    alex.goto(i, t_steps - j)
                    alex.dot('green')
                else:
                    alex.goto(i, t_steps - j)
                    alex.dot('red')
    wn.update()
    wn.exitonclick()


def plot_ave_spd_v_rho(p_l, n_cells, t_steps, d_rho):
    print(p_l)
    x_min = 0.0
    x_max = 1.0
    y_max = 6.0
    y_min = 0.0
    wn.setworldcoordinates(x_min, y_min, x_max, y_max)
    wn.tracer(0)
    alex.up()
    alex.goto(x_min, y_max)
    for p in p_l:
        alex.up()
        for i in range(int(1/d_rho)-1):
            rho = (i+1)*d_rho
            n_veh = int(n_cells * rho)
            loc_lists, spd_lists = simulate_flow(rho, p, n_cells, t_steps)
            spd_l = spd_lists[-1]
            ave_spd = sum(spd_l)/n_veh
            alex.goto(rho, ave_spd)
            alex.down()
            alex.dot()
    wn.update()
    wn.exitonclick()


def plot_gflow_v_rho(p_l, n_cells, t_steps, d_rho):
    x_min = 0.0
    x_max = 1.0
    y_max = 1.0
    y_min = 0.0
    wn.setworldcoordinates(x_min, y_min, x_max, y_max)
    wn.tracer(0)
    alex.up()
    alex.goto(x_min, y_min)
    for p in p_l:
        alex.up()
        for i in range(int(1 / d_rho)-1):
            rho = (i + 1) * d_rho
            n_veh = int(n_cells * rho)
            loc_lists, spd_lists = simulate_flow(rho, p, n_cells, t_steps)
            spd_l = spd_lists[-1]
            ave_spd = sum(spd_l) / n_veh
            g_flow = ave_spd * rho
            alex.goto(rho, g_flow)
            alex.down()
            alex.dot()
    wn.update()
    wn.exitonclick()


def main():
    rho = 0.3
    p = 0.5
    n_cells = 200
    t_steps = 200
    plot_flow(rho, p, n_cells, t_steps)
    #p_l = [0.0, 0.1, 0.3, 0.7, 0.9]
    #d_rho = 0.01
    #plot_ave_spd_v_rho(p_l, n_cells, t_steps, d_rho)
    #plot_gflow_v_rho(p_l, n_cells, t_steps, d_rho)


main()



TclError: no display name and no $DISPLAY environment variable